# Исследование данных и первичная подготовка (EDA)

**Цель ноутбука.**  
Проверить целостность и структуру таблиц `user`, `post`, `feed`, понять семантику таргета, выявить базовые аномалии и подготовить первый сэмпл событий для последующих шагов (feature engineering, обучение модели).

**Исходные данные**  
- `user.csv` — профиль пользователя (демография, устройство, источник);  
- `post.csv` — карточка поста (текст, тема);  
- `feed_data.csv` — логи событий `view/like` с `timestamp` и `target`.

**Данные после обработки/извлечения**  
- `feed_sampled.csv` — отобранные события просмотров с таргетом.

**Важно / предположения:**  
- В этом EDA **не вносим изменений в код**, только интерпретация и фиксация решений.  
- Поле `exp_group` из `user` используется **только для анализа**; для A/B-разбиения в сервисе будет применяться хэш-сплит (md5+соль).  
- Таргет (`target = 1`) определён **для событий `view`**, если «почти сразу» был `like`.

**Проверки качества данных:**  
- Дубликаты и пропуски; согласованность ключей (`user_id`, `post_id`);  
- Диапазоны числовых полей (`age`);  
- Адекватность временных полей (`timestamp`).


На этом этапе подключаемся к источникам, загружаем данные в `pandas`, проверяем целостность и распределения. Особое внимание — на роль поля `target`, семантику действий (`view`/`like`) и корректность временных полей.

In [ ]:
# Импортируем базовый стек анализа и настраиваем отображение

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 80, 'display.max_columns', 50)

In [2]:
import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5

mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

In [ ]:
# Загружаем локальные выгрузки `user`, `post`, `feed`
user = pd.read_csv('user.csv')
post = pd.read_csv('post.csv')
feed = pd.read_csv('feed_data.csv')

Изучим таблицу **user**.

In [4]:
user.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [5]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    163205 non-null  int64 
 1   gender     163205 non-null  int64 
 2   age        163205 non-null  int64 
 3   country    163205 non-null  object
 4   city       163205 non-null  object
 5   exp_group  163205 non-null  int64 
 6   os         163205 non-null  object
 7   source     163205 non-null  object
dtypes: int64(4), object(4)
memory usage: 10.0+ MB


In [6]:
user.describe()

,user_id,gender,age,exp_group
count,163205.000000,163205.000000,163205.000000,163205.000000
mean,85070.371759,0.551331,27.195405,1.997598
std,48971.639950,0.497360,10.239158,1.413644
min,200.000000,0.000000,14.000000,0.000000
25%,41030.000000,0.000000,19.000000,1.000000
50%,85511.000000,1.000000,24.000000,2.000000
75%,127733.000000,1.000000,33.000000,3.000000
max,168552.000000,1.000000,95.000000,4.000000


In [7]:
user.describe(include='object')

,country,city,os,source
count,163205,163205,163205,163205
unique,11,3915,2,2
top,Russia,Moscow,Android,ads
freq,143035,21874,105972,101685


In [8]:
user.shape

(163205, 8)

In [9]:
user.duplicated().sum()

0

In [10]:
user.nunique()

user_id      163205
gender            2
age              76
country          11
city           3915
exp_group         5
os                2
source            2
dtype: int64

In [11]:
np.sort(user['age'].unique())

array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
       48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
       65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
       82, 83, 84, 85, 86, 87, 92, 95], dtype=int64)

In [12]:
user['age'].value_counts().sort_index()

14     2396
15     5374
16     6420
17     7785
18     9034
19     9802
20    10280
21    10139
22     9049
23     8120
24     6795
25     5853
26     5116
27     4528
28     4193
29     4074
30     3952
31     3767
32     3519
33     3395
34     3364
35     3261
36     3089
37     2708
38     2674
39     2432
40     2274
41     2099
42     1965
43     1782
44     1610
45     1502
46     1319
47     1219
48     1077
49      950
50      894
51      808
52      673
53      584
54      492
55      445
56      380
57      308
58      283
59      245
60      210
61      142
62      156
63      107
64      119
65       78
66       77
67       55
68       42
69       27
70       32
71       23
72       23
73       16
74       16
75        6
76       13
77        8
78        7
79        3
80        2
81        2
82        3
83        1
84        3
85        1
86        1
87        2
92        1
95        1
Name: age, dtype: int64

In [ ]:
# Удаляем `exp_group` из `user` перед дальнейшей работой с фичами
# В сервисе A/B-группа должна определяться независимо (хэш-функцией),
# а не из этой колонки
user = user.drop('exp_group', axis=1)

Изучим таблицу **post**.

In [14]:
post.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [15]:
post.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   post_id  7023 non-null   int64 
 1   text     7023 non-null   object
 2   topic    7023 non-null   object
dtypes: int64(1), object(2)
memory usage: 164.7+ KB


In [16]:
post.describe()

,post_id
count,7023.000000
mean,3666.533817
std,2109.613383
min,1.000000
25%,1849.500000
50%,3668.000000
75%,5492.000000
max,7319.000000


In [17]:
post.describe(include='object')

,text,topic
count,7023,7023
unique,6924,7
top,Microsoft gets the blogging bug\n\nSoftware gi...,movie
freq,2,3000


In [18]:
post.shape

(7023, 3)

In [19]:
post.duplicated().sum()

0

Изучим таблицу **feed**.

In [20]:
feed['user_id'].nunique()

163205

In [21]:
feed.head()

,timestamp,user_id,post_id,action,target
0,2021-10-21 11:16:14,51533,1784,view,1
1,2021-10-21 11:18:16,51533,1784,like,0
2,2021-10-21 11:18:18,51533,1344,view,0
3,2021-10-21 11:19:03,51533,1911,view,0
4,2021-10-21 11:20:33,51533,1622,view,0


In [22]:
feed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76892800 entries, 0 to 76892799
Data columns (total 5 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  object
 1   user_id    int64 
 2   post_id    int64 
 3   action     object
 4   target     int64 
dtypes: int64(3), object(2)
memory usage: 2.9+ GB


In [23]:
feed.describe()

,user_id,post_id,target
count,7.689280e+07,7.689280e+07,7.689280e+07
mean,8.508015e+04,3.397824e+03,1.067245e-01
std,4.897304e+04,2.095346e+03,3.087627e-01
min,2.000000e+02,1.000000e+00,0.000000e+00
25%,4.103000e+04,1.528000e+03,0.000000e+00
50%,8.550900e+04,3.195000e+03,0.000000e+00
75%,1.277370e+05,5.207000e+03,0.000000e+00
max,1.685520e+05,7.319000e+03,1.000000e+00


In [24]:
feed.describe(include='object')

,timestamp,action
count,76892800,76892800
unique,2598589,2
top,2021-10-16 16:51:22,view
freq,192,68686455


In [25]:
feed.shape

(76892800, 5)

In [26]:
feed.isna().sum()

timestamp    0
user_id      0
post_id      0
action       0
target       0
dtype: int64

In [27]:
feed['action'].value_counts()

view    68686455
like     8206345
Name: action, dtype: int64

In [28]:
feed.duplicated().sum()

0

Исключить строки с action = `'like'`

In [ ]:
# Ниже оставляем в логе только события `view` (таргет определён на них).
# Для ускорения экспериментов берём первые 6 млн строк
filtered_feed = feed[feed['action'] == 'view']

In [ ]:
# Оставить только нужные столбцы
filtered_feed = filtered_feed[['user_id', 'post_id', 'timestamp', 'target']]

In [33]:
filtered_feed = filtered_feed[:6_000_000]
filtered_feed.shape

(6000000, 4)

In [34]:
filtered_feed['timestamp'] = pd.to_datetime(filtered_feed['timestamp'])

In [35]:
filtered_feed.head()

,user_id,post_id,timestamp,target
0,51533,1784,2021-10-21 11:16:14,1
2,51533,1344,2021-10-21 11:18:18,0
3,51533,1911,2021-10-21 11:19:03,0
4,51533,1622,2021-10-21 11:20:33,0
5,51533,1211,2021-10-21 11:21:33,0


In [ ]:
# Сохраняем промежуточный артефакт `feed_sampled.csv` для ускорения последующих ноутбуков
filtered_feed.to_csv('feed_sampled.csv', index=False)

## Итоги EDA и следующие шаги

**Что проверили в этом ноутбуке:**
- Структура и базовые распределения в `user`, `post`, `feed`;
- Семантика таргета и событий (`view`/`like`);
- Отсутствие дублей по ключам (`user_id`, `post_id`);
- Приведение времени к `datetime` и подготовка рабочего сэмпла.

**Ограничения текущего подхода:**
- Отбор «первых 6 млн строк» может быть нерепрезентативным по времени;
- Пути/типы/константы заданы в коде; в проде лучше вынести в конфиг.
